In [1]:
## original code from https://biapol.github.io/Image-data-science-with-Python-and-Napari-EPFL2022/day2d_image_segmentation/12_Segmentation_3D.html
## env setting to see https://github.com/clEsperanto/pyclesperanto_prototype/tree/master

In [1]:
from skimage.io import imread
from skimage.io import imsave
from skimage import filters
from skimage import morphology
from pyclesperanto_prototype import imshow
import pyclesperanto_prototype as cle
import matplotlib.pyplot as plt

In [2]:
import napari
from napari.utils import nbscreenshot

In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from skimage import measure

In [4]:
import os
os.getcwd()

'/groups/tanaka/People/current/jiwang/projects/RA_competence/scripts'

In [5]:
cle.available_device_names()

['Oclgrind Simulator']

In [6]:
# For 3D processing, powerful graphics
# processing units might be necessary
cle.select_device('TX')

/groups/tanaka/People/current/jiwang/local/anaconda3/envs/image_py3.9/lib/python3.9/site-packages/pyclesperanto_prototype/_tier0/_device.py:77: UserWarning: No OpenCL device found with TX in their name. Using Oclgrind Simulator instead.
  warnings.warn(f"No OpenCL device found with {name} in their name. Using {device.name} instead.")


<Oclgrind Simulator on Platform: Oclgrind (2 refs)>

In [7]:
def show(image_to_show, labels=False):
    """
    This function generates three projections: in X-, Y- and Z-direction and shows them.
    """
    projection_x = cle.maximum_x_projection(image_to_show)
    projection_y = cle.maximum_y_projection(image_to_show)
    projection_z = cle.maximum_z_projection(image_to_show)

    fig, axs = plt.subplots(1, 3, figsize=(15, 15))
    cle.imshow(projection_x, plot=axs[0], labels=labels)
    cle.imshow(projection_y, plot=axs[1], labels=labels)
    cle.imshow(projection_z, plot=axs[2], labels=labels)
    plt.show()


In [8]:
#################################################################
#################################################################
## RA control samples to check the FoxA2/Pax6 ratios
#################################################################
#################################################################

In [14]:
imageDir = "../images_data/d4_WT"

In [15]:
outDir = "../images_data/results/test_WT_FoxA2_pct"
if not os.path.exists(outDir):
    os.mkdir(outDir)
#os.path.exists(outDir)

In [11]:
fileName = '20220726_d4_10x_RA-18h_E2_121_1745_isotropic'

In [14]:
mask = imread(os.path.join(imageDir, str(fileName + "_C4_CystMask.tiff"))) # mask segmeted cysts
C3 = imread(os.path.join(imageDir, str(fileName + "_C3.tif"))) # FoxA2, nuclei
C2 = imread(os.path.join(imageDir, str(fileName + "_C2.tif"))) # Pax6, nuclei

#C3 = C3 / np.quantile(C3, 0.95)
#C2 = C2 / np.quantile(C2, 0.95)

In [18]:
from skimage import measure
import pandas as pd
from skimage.filters import threshold_otsu, rank
import pyclesperanto_prototype as cle
from skimage import filters
from skimage.filters import try_all_threshold
import napari_simpleitk_image_processing as nsitk

## try the local thresholding 
## https://scikit-image.org/docs/stable/auto_examples/applications/plot_thresholding_guide.html
## threshold_local or skimage.filters.rank.otsu

In [ ]:
labels_mask, nb_cyst = measure.label(mask, return_num = True)

In [ ]:
nb_cyst

27

In [82]:
cyst_index = np.where(labels_mask == 2)
cyst_index[0].size
#genotype_wt = C3[cyst_index]
#genotype_ko = C2[cyst_index]

40508

In [83]:
foxa2 = C3[cyst_index]
pax6 = C2[cyst_index]

foxa2_blur = filters.gaussian(foxa2, sigma=1.0)
pax6_blur = filters.gaussian(pax6, sigma=1.0)

In [84]:
cutoff_foxa2 = threshold_otsu(foxa2)
cutoff_pax6 = threshold_otsu(pax6)
cutoff_foxa2_blur = threshold_otsu(foxa2_blur)
cutoff_pax6_blur = threshold_otsu(pax6_blur)
print(cutoff_foxa2, cutoff_pax6, cutoff_foxa2_blur, cutoff_pax6_blur)

2.0603932584269664 12.28093274111675 2.094923647915625 12.183763005104478


In [85]:
xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6))
yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print(xx[0].size/(xx[0].size + yy[0].size + zz[0].size))

xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6))
yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
zz1 = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6) & (foxa2 > pax6))
zz2 = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6) & (foxa2 < pax6))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print((xx[0].size + zz1[0].size)/(xx[0].size + yy[0].size + zz1[0].size + zz2[0].size))
#print(zz1[0].size, zz2[0].size)

xx = np.where((foxa2_blur > cutoff_foxa2_blur) & (pax6_blur < cutoff_pax6_blur))
yy = np.where((foxa2_blur < cutoff_foxa2_blur ) & (pax6_blur > cutoff_pax6_blur))
zz = np.where((foxa2_blur > cutoff_foxa2_blur ) & (pax6_blur > cutoff_pax6_blur))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print(xx[0].size/(xx[0].size + yy[0].size + zz[0].size))

## either one positive 
#sels = np.where((foxa2 > cutoff_foxa2) | (pax6 > cutoff_pax6))
#pax6_sels = pax6[sels]
#foxa2_sels = foxa2[sels]
# specify the FoxA2-KO and Pax6-KO genotype voxels 
#index_foxa2 = np.where(foxa2_sels >= pax6_sels)
#index_pax6 = np.where(foxa2_sels < pax6_sels)
#print(index_foxa2[0].size/(index_foxa2[0].size + index_pax6[0].size))

0.1438619484491044
0.1438619484491044
0.10499820208558072


In [86]:
cutoff_foxa2 = filters.threshold_isodata(foxa2)
cutoff_pax6 = filters.threshold_isodata(pax6)
cutoff_foxa2_blur = filters.threshold_isodata(foxa2_blur)
cutoff_pax6_blur = filters.threshold_isodata(pax6_blur)
print(cutoff_foxa2, cutoff_pax6, cutoff_foxa2_blur, cutoff_pax6_blur)
xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6))
yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print(xx[0].size/(xx[0].size + yy[0].size + zz[0].size))

xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6))
yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
zz1 = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6) & (foxa2 > pax6))
zz2 = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6) & (foxa2 < pax6))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print((xx[0].size + zz1[0].size)/(xx[0].size + yy[0].size + zz1[0].size + zz2[0].size))
#print(zz1[0].size, zz2[0].size)

xx = np.where((foxa2_blur > cutoff_foxa2_blur) & (pax6_blur < cutoff_pax6_blur))
yy = np.where((foxa2_blur < cutoff_foxa2_blur ) & (pax6_blur > cutoff_pax6_blur))
zz = np.where((foxa2_blur > cutoff_foxa2_blur ) & (pax6_blur > cutoff_pax6_blur))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print(xx[0].size/(xx[0].size + yy[0].size + zz[0].size))

2.0603932584269664 12.28093274111675 2.067557908817686 12.183763005104478
0.1438619484491044
0.1438619484491044
0.11138878971795788


In [87]:
cutoff_foxa2 = filters.threshold_li(foxa2)
cutoff_pax6 = filters.threshold_li(pax6)
cutoff_foxa2_blur = filters.threshold_li(foxa2_blur)
cutoff_pax6_blur = filters.threshold_li(pax6_blur)
print(cutoff_foxa2, cutoff_pax6, cutoff_foxa2_blur, cutoff_pax6_blur)

1.8810858637601733 9.855629027130883 1.9019925167853553 10.034888387555059


In [89]:
xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6))
yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print(xx[0].size/(xx[0].size + yy[0].size + zz[0].size))
#print(zz[0].size)
print(xx[0].size/(xx[0].size + yy[0].size))

xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6))
yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
zz1 = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6) & (foxa2 > pax6))
zz2 = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6) & (foxa2 < pax6))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print((xx[0].size + zz1[0].size)/(xx[0].size + yy[0].size + zz1[0].size + zz2[0].size))
#print(zz1[0].size, zz2[0].size)


xx = np.where((foxa2_blur > cutoff_foxa2_blur) & (pax6_blur < cutoff_pax6_blur))
yy = np.where((foxa2_blur < cutoff_foxa2_blur ) & (pax6_blur > cutoff_pax6_blur))
zz = np.where((foxa2_blur > cutoff_foxa2_blur ) & (pax6_blur > cutoff_pax6_blur))
#print(zz[0].size/(xx[0].size + yy[0].size + zz[0].size))
print(xx[0].size/(xx[0].size + yy[0].size + zz[0].size))


0.08885737917995983
0.5651302605210421
0.08885737917995983
0.07236444265269099


In [16]:
import pandas as pd
from skimage import measure
from skimage.filters import threshold_otsu, rank

data = {"image": [], "cyst_index": [], "cyst_size": [],  
        "cutoff_otsu_cyst_foxa2": [],  "cutoff_otsu_cyst_pax6": [], "nb_foxa2_otsu_cyst": [], "nb_pax6_otsu_cyst": [],  "nb_double_otsu_cyst": [],
        "cutoff_otsu_global_foxa2": [],  "cutoff_otsu_global_pax6": [], "nb_foxa2_otsu_global": [], "nb_pax6_otsu_global": [],  "nb_double_otsu_global": [],
        "cutoff_li_cyst_foxa2": [],  "cutoff_li_cyst_pax6": [], "nb_foxa2_li_cyst": [], "nb_pax6_li_cyst": [],  "nb_double_li_cyst": [],
        "cutoff_mean_cyst_foxa2": [],  "cutoff_mean_cyst_pax6": [], "nb_foxa2_mean_cyst": [], "nb_pax6_mean_cyst": [],  "nb_double_mean_cyst": [],
        "cutoff_li_global_foxa2": [],  "cutoff_li_global_pax6": [], "nb_foxa2_li_global": [], "nb_pax6_li_global": [],  "nb_double_li_global": [],
        "nb_foxa2_localThreshold": [], "nb_pax6_localThreshold": [],  "nb_double_localThreshold": []
        
       }
df = pd.DataFrame(data)
df

,image,cyst_index,cyst_size,cutoff_otsu_cyst_foxa2,cutoff_otsu_cyst_pax6,nb_foxa2_otsu_cyst,nb_pax6_otsu_cyst,nb_double_otsu_cyst,cutoff_otsu_global_foxa2,cutoff_otsu_global_pax6,...,nb_pax6_mean_cyst,nb_double_mean_cyst,cutoff_li_global_foxa2,cutoff_li_global_pax6,nb_foxa2_li_global,nb_pax6_li_global,nb_double_li_global,nb_foxa2_localThreshold,nb_pax6_localThreshold,nb_double_localThreshold


In [17]:
for nm in os.listdir(imageDir):
    if "_C4_CystMask.tiff" in nm:
        #print(nm)
        fileName = nm.replace('_C4_CystMask.tiff','')
        #files += [nm]
        print(fileName)
        #make_composite_C5(newName, ImageDir, outDir)
        
        mask = imread(os.path.join(imageDir, str(fileName + "_C4_CystMask.tiff"))) # mask segmeted cysts
        C3 = imread(os.path.join(imageDir, str(fileName + "_C3.tif"))) # FoxA2, nuclei
        C2 = imread(os.path.join(imageDir, str(fileName + "_C2.tif"))) # Pax6, nuclei
        
        #C3_blur = filters.gaussian(C3, sigma=1.0)
        #C2_blur = filters.gaussian(C2, sigma=1.0)
        
        #C3 = C3 / np.quantile(C3, 0.99)
        #C2 = C2 / np.quantile(C2, 0.99)
        
        cutoff_global_otsu_foxa2 = threshold_otsu(C3)
        cutoff_global_otsu_pax6 = threshold_otsu(C2)
        cutoff_global_li_foxa2 = filters.threshold_li(C3)
        cutoff_global_li_pax6 = filters.threshold_li(C2)
        
        #image = ski.data.page()

        block_size = 51
        local_thresh_C3 = filters.threshold_local(C3, block_size, offset=0)
        binary_local_C3 = C3 > local_thresh_C3
        local_thresh_C2 = filters.threshold_local(C2, block_size, offset=0)
        binary_local_C2 = C2 > local_thresh_C2
        
        
        #mask = imread(os.path.join(CPoutDir, str(fileName + "_C4_CystMask.tiff"))) # mask segmeted cysts
        #C1 = imread(os.path.join(CPoutDir, str(fileName + "_C3_FoxA2mask.tiff"))) # FoxA2, nuclei
        #C2 = imread(os.path.join(CPoutDir, str(fileName + "_C2_Pax6mask.tiff"))) # Pax6, nuclei
        
        labels_mask, nb_cyst = measure.label(mask, return_num = True)

        for i in range(nb_cyst+1):
            if i > 0:
                #print(i) # prints: -1, 0, 1, 2, 3, 4,
                cyst_index = np.where(labels_mask == i)
                
                keep = [fileName, i, cyst_index[0].size]
                
                foxa2 = C3[cyst_index]
                pax6 = C2[cyst_index]
                
                #foxa2_blur = C3_blur[cyst_index]
                #pax6_blur = C2_blur[cyst_index]
                
                ## cutoff of FoxA2 and Pax6 in each cyst with otsu 
                cutoff_foxa2 = threshold_otsu(foxa2)
                cutoff_pax6 = threshold_otsu(pax6)
                
                ## define the FoxA2+ only, Pax6+ only, and double positive with cyst otsu 
                xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6)) #  
                yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                
                keep = keep + [cutoff_foxa2, cutoff_pax6, xx[0].size, yy[0].size, zz[0].size]
                
                xx = np.where((foxa2 > cutoff_global_otsu_foxa2) & (pax6 < cutoff_global_otsu_pax6)) #  
                yy = np.where((foxa2 < cutoff_global_otsu_foxa2) & (pax6 > cutoff_global_otsu_pax6))
                zz = np.where((foxa2 > cutoff_global_otsu_foxa2) & (pax6 > cutoff_global_otsu_pax6))
                
                keep = keep + [cutoff_global_otsu_foxa2, cutoff_global_otsu_pax6, xx[0].size, yy[0].size, zz[0].size]
                
                ## cutoff of FoxA2 and Pax6 in each cyst with li 
                cutoff_foxa2 = filters.threshold_li(foxa2)
                cutoff_pax6 = filters.threshold_li(pax6)
                
                xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6)) #  
                yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                
                keep = keep + [cutoff_foxa2, cutoff_pax6, xx[0].size, yy[0].size, zz[0].size]
                
                ## cutoff of FoxA2 and Pax6 in each cyst with mean thresholding
                cutoff_foxa2 = filters.threshold_mean(foxa2)
                cutoff_pax6 = filters.threshold_mean(pax6)
                xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6)) #  
                yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                
                keep = keep + [cutoff_foxa2, cutoff_pax6, xx[0].size, yy[0].size, zz[0].size]
                
                xx = np.where((foxa2 > cutoff_global_li_foxa2) & (pax6 < cutoff_global_li_pax6)) #  
                yy = np.where((foxa2 < cutoff_global_li_foxa2) & (pax6 > cutoff_global_li_pax6))
                zz = np.where((foxa2 > cutoff_global_li_foxa2) & (pax6 > cutoff_global_li_pax6))
                
                
                keep = keep + [cutoff_global_li_foxa2, cutoff_global_li_pax6, xx[0].size, yy[0].size, zz[0].size]
                
                ## local thresholding
                #foxa2 = binary_local_C3[cyst_index]
                #pax6 = binary_local_C2[cyst_index]
                #xx = np.where((foxa2 > 0) & (pax6 < 1)) #  
                #yy = np.where((foxa2 < 1) & (pax6 > 0))
                #zz = np.where((foxa2 > 0) & (pax6 > 0))
                
                ## mixture of thresholding
                cutoff_foxa2 = filters.threshold_mean(foxa2)
                cutoff_pax6 = threshold_otsu(pax6)
                
                xx = np.where((foxa2 > cutoff_foxa2) & (pax6 < cutoff_pax6)) #  
                yy = np.where((foxa2 < cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                zz = np.where((foxa2 > cutoff_foxa2 ) & (pax6 > cutoff_pax6))
                
                keep = keep + [xx[0].size, yy[0].size, zz[0].size]
                
                df.loc[len(df)] = keep
                

20220726_d4_10x_RA-18h_E2_121_1745_isotropic
20220726_d4_10x_RA-18h_E2_122_1746_isotropic
20220726_d4_10x_RA-18h_E2_123_1747_isotropic
20220726_d4_10x_RA-18h_E2_124_1748_isotropic
20220726_d4_10x_RA-18h_F2_161_1785_isotropic
20220726_d4_10x_RA-18h_F2_162_1786_isotropic
20220726_d4_10x_RA-18h_F2_163_1787_isotropic
20220726_d4_10x_RA-18h_F2_164_1788_isotropic
20220726_d4_10x_RA-18h_G2_201_1825_isotropic
20220726_d4_10x_RA-18h_G2_202_1826_isotropic
20220726_d4_10x_RA-18h_G2_203_1827_isotropic
20220726_d4_10x_RA-18h_G2_204_1828_isotropic


In [18]:
outDir
df.to_csv(os.path.join(outDir, "wt_FoxA2_Pax6_voxel_counts_global_cyst_otsu_li_mean_mixingThreshold.csv"), index=True, header=True)

In [ ]:
df = pd.read_csv(os.path.join(outDir, "wt_FoxA2_Pax6.csv"), index_col = 0)
df

In [ ]:
df.plot(kind='scatter', x = 'total', y =  'pct_foxa2')

In [ ]:
df.plot(kind='scatter', x = 'cyst_size', y =  'pct_foxa2')

In [ ]:
sel = np.where(df[["total"]] > 5000)
df = df.iloc[sel[0], ]
df.plot(kind='scatter', x = 'cyst_size', y =  'pct_foxa2')

In [ ]:
np.mean(df[['pct_foxa2']])

In [ ]:
np.std(df[['pct_foxa2']])

In [ ]:
#rng = np.random.RandomState(10)  # deterministic random data
#a = np.hstack((rng.normal(size=1000),
#               rng.normal(loc=5, scale=2, size=1000)))
a = df[['pct_foxa2']]
plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
#################################################################
#################################################################
## quantify the Olig2 in WT samples with RA 
#################################################################
#################################################################

In [9]:
import pandas as pd
from skimage import measure
from skimage.morphology import disk
#from skimage.filters import threshold_otsu, rank
import pyclesperanto_prototype as cle
from skimage import filters
from skimage.filters import try_all_threshold
import napari_simpleitk_image_processing as nsitk

In [10]:
imageDir = "../images_data/d6_10x_WT-Olig2"
outDir = "../images_data/results/WT_d6_stain_DVpatterning"

if not os.path.exists(outDir):
    os.mkdir(outDir)
#os.path.exists(outDir)

In [11]:
#fileName = '241023_10x_d3-2umZ_1-FS-PG_RA_FA2-Px6_C7_13_01_isotropic'
fileName = "20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_E2_149_409_isotropic"

In [14]:
mask = imread(os.path.join(imageDir, str(fileName + "_C4_AllCystMask.tiff"))) # mask segmeted cysts
C3 = imread(os.path.join(imageDir, str(fileName + "_C3.tif"))) # Olig2

In [17]:
print(np.quantile(C3, 0.95), np.max(C3))

340.0 6085


In [18]:
labels_mask, nb_cyst = measure.label(mask, return_num = True)
nb_cyst

34

In [22]:
cutoff_pax6 = filters.threshold_otsu(C3)
cutoff_pax62 = filters.threshold_li(C3)
print(cutoff_pax6, cutoff_pax62)

296 158.42657821365125


In [ ]:
cutoff_marker2 = filters.threshold_multiotsu(C3, classes=3, nbins = 512)
cutoff_marker3 = filters.threshold_multiotsu(C3, classes=4, nbins = 512)
print(cutoff_marker2, cutoff_marker3)

In [ ]:
import pyclesperanto_prototype as cle
marker_binary = C3 > 485
## visualize the 3D images https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/16_3d_image_visualization/inspecting_3d_images.html
another_slice = cle.create_2d_xy(marker_binary)
cle.copy_slice(marker_binary, another_slice, slice_index = 46)
# show result
cle.imshow(another_slice)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 5))
ax = axes.ravel()

ax[0].hist(C1.ravel(), bins=2000)
ax[0].set_xlim([0, 3])
ax[0].axvline(cutoff_foxa2, color='r')

ax[1].hist(C4.ravel(), bins=512)
ax[1].set_xlim([0, 300])
ax[1].axvline(cutoff_pax6, color='r')
ax[1].axvline(cutoff_marker2[0], color='b')
ax[1].axvline(cutoff_marker2[1], color='b')
ax[1].axvline(cutoff_marker3[0], color='g')
ax[1].axvline(cutoff_marker3[1], color='g')
ax[1].axvline(cutoff_marker3[2], color='g')

In [23]:
import pandas as pd
from skimage import measure
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
from skimage import measure
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
import pyclesperanto_prototype as cle
from skimage import filters
from skimage.filters import try_all_threshold
import napari_simpleitk_image_processing as nsitk

from skimage import measure

data = {"image": [], "cyst_index": [], "cyst_size": [], 
        "nb_marker_global": [],  "cutoff_marker_image": [],
         "nb_marker_cyst": [], "cutoff_marker": []
       }

df = pd.DataFrame(data)
df

,image,cyst_index,cyst_size,nb_marker_global,cutoff_marker_image,nb_marker_cyst,cutoff_marker


In [24]:
for nm in os.listdir(imageDir):
    if "_C4_AllCystMask.tiff" in nm:
        #print(nm)
        fileName = nm.replace('_C4_AllCystMask.tiff','')
        #files += [nm]
        #print(fileName)
        #make_composite_C5(newName, ImageDir, outDir)
        if "_mix2_" in fileName:
            mask = imread(os.path.join(imageDir, str(fileName + "_C4_AllCystMask.tiff"))) # mask segmeted cysts
            C3 = imread(os.path.join(imageDir, str(fileName + "_C3.tif"))) # Pax6-KO genotype 
            
            labels_mask, nb_cyst = measure.label(mask, return_num = True)
            cutoff_marker_image = 780
            
            print(fileName, nb_cyst)
        else:
            #cutoff_pax6_image = filters.threshold_multiotsu(C4, classes=3, nbins = 512)[1]
            continue
            
        for i in range(nb_cyst+1):
            if i > 0:
                cyst_index = np.where(labels_mask == i)
                
                marker = C3[cyst_index]
                
                index_marker_global = np.where(marker > cutoff_marker_image)
                #index_pax6_global = np.where(pax6 > cutoff_pax6_image)
                
                #cutoff_fako = filters.threshold_otsu(genotype_fako)
                #cutoff_pxko = filters.threshold_otsu(genotype_pxko)
                cutoff_marker = filters.threshold_otsu(marker)
                #cutoff_pax6 = filters.threshold_otsu(pax6)
                
                index_marker_cyst = np.where(marker > cutoff_marker)
                #index_pax6_cyst = np.where(pax6 > cutoff_pax6)
                
                
                #quantile95_fako = np.quantile(genotype_fako, 0.95)
                #quantile95_pxko = np.quantile(genotype_pxko, 0.95)
                #quantile95_foxa2 = np.quantile(foxa2, 0.95)
                #quantile95_pax6 = np.quantile(pax6, 0.95)
                
                if (cyst_index[0].size) > 1000:
                    df.loc[len(df)] = [fileName, i,  
                                        cyst_index[0].size,
                                        index_marker_global[0].size, cutoff_marker_image, 
                                        index_marker_cyst[0].size, cutoff_marker
                                        ]
                                    

20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_E1_145_406_isotropic 46
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_E1_146_407_isotropic 33
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_E1_147_408_isotropic 47
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_F1_193_442_isotropic 50
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_F1_194_443_isotropic 44
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_F1_195_444_isotropic 37
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_G1_241_478_isotropic 42
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_G1_242_479_isotropic 35
20220309_FoxA2KOpatterning_10x_Wt_NoRA_mix2_G1_243_480_isotropic 48
20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_E2_149_409_isotropic 34
20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_E2_150_410_isotropic 24
20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_E2_151_411_isotropic 30
20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_F2_197_445_isotropic 23
20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_F2_198_446_isotropic 26
20220309_FoxA2KOpatterning_10x_Wt_RA_mix2_F2_199_447_isotr

In [26]:
df.to_csv(os.path.join(outDir, "WT_d6_cyst_size_manualOlig2global_v2.csv"), index=True, header=True)

In [ ]:
################################################
##
## quantify the FoxA2+, Pax+ and double+ after Illastik pixel classification
## 
################################################

In [8]:
from skimage import measure
import pandas as pd
from skimage.filters import threshold_otsu, rank
import pyclesperanto_prototype as cle
from skimage import filters
from skimage.filters import try_all_threshold
import napari_simpleitk_image_processing as nsitk

## try the local thresholding 
## https://scikit-image.org/docs/stable/auto_examples/applications/plot_thresholding_guide.html
## threshold_local or skimage.filters.rank.otsu

In [9]:
imageDir = "../images_data/d4_WT"

In [10]:
outDir = "../images_data/results/test_WT_FoxA2_pct_ilastik"
if not os.path.exists(outDir):
    os.mkdir(outDir)
#os.path.exists(outDir)

In [19]:
fileName = "20220726_d4_10x_RA-18h_G2_201_1825_isotropic"

print(fileName)

mask = imread(os.path.join(imageDir, str(fileName + "_C4_CystMask.tiff"))) # mask segmeted cysts
        
C3 = imread(os.path.join(imageDir, str("FoxA2/" + fileName + "_C3_Probabilities.tiff"))) # FoxA2, nuclei
C2 = imread(os.path.join(imageDir, str("Pax6/" + fileName + "_C2_Probabilities.tiff"))) # Pax6, nuclei
        
labels_mask, nb_cyst = measure.label(mask, return_num = True)
C3 = C3[:, 0, :, :]
C2 = C2[:, 0, :, :]

20220726_d4_10x_RA-18h_G2_201_1825_isotropic


In [24]:
nb_cyst

24

In [20]:
#C3_0 = C3[:, 0, :, :]

In [25]:
#C3_0.shape, np.max(C3_0)

In [26]:
C3.shape, mask.shape, C2.shape

((91, 499, 499), (91, 499, 499), (91, 499, 499))

In [11]:
import pandas as pd
from skimage import measure
from skimage.filters import threshold_otsu, rank

data = {"image": [], "cyst_index": [], "cyst_size": [],  
        "nb_foxa2_cyst": [], "nb_pax6_cyst": [],  "nb_double_cyst": []
       }
df = pd.DataFrame(data)
df

,image,cyst_index,cyst_size,nb_foxa2_cyst,nb_pax6_cyst,nb_double_cyst


In [12]:
for nm in os.listdir(imageDir):
    if "_C4_CystMask.tiff" in nm:
        #print(nm)
        fileName = nm.replace('_C4_CystMask.tiff','')
        #files += [nm]
        print(fileName)
        #make_composite_C5(newName, ImageDir, outDir)
        
        mask = imread(os.path.join(imageDir, str(fileName + "_C4_CystMask.tiff"))) # mask segmeted cysts
        
        C3 = imread(os.path.join(imageDir, str("FoxA2/" + fileName + "_C3_Probabilities.tiff"))) # FoxA2, nuclei
        C2 = imread(os.path.join(imageDir, str("Pax6/" + fileName + "_C2_Probabilities.tiff"))) # Pax6, nuclei
        C3 = C3[:, 0, :, :]
        C2 = C2[:, 0, :, :]
        
        labels_mask, nb_cyst = measure.label(mask, return_num = True)

        for i in range(nb_cyst+1):
            if i > 0:
                #print(i) # prints: -1, 0, 1, 2, 3, 4,
                cyst_index = np.where(labels_mask == i)
                
                keep = [fileName, i, cyst_index[0].size]
                
                foxa2 = C3[cyst_index]
                pax6 = C2[cyst_index]
                
                cutoff_prob = 0.5
                ## define the FoxA2+ only, Pax6+ only, and double positive with cyst otsu 
                xx = np.where((foxa2 > cutoff_prob) & (pax6 <= cutoff_prob)) #  
                yy = np.where((foxa2 <= cutoff_prob ) & (pax6 > cutoff_prob))
                zz = np.where((foxa2 > cutoff_prob ) & (pax6 > cutoff_prob))
                
                keep = keep + [xx[0].size, yy[0].size, zz[0].size]
                
                df.loc[len(df)] = keep

20220726_d4_10x_RA-18h_E2_121_1745_isotropic
20220726_d4_10x_RA-18h_E2_122_1746_isotropic
20220726_d4_10x_RA-18h_E2_123_1747_isotropic
20220726_d4_10x_RA-18h_E2_124_1748_isotropic
20220726_d4_10x_RA-18h_F2_161_1785_isotropic
20220726_d4_10x_RA-18h_F2_162_1786_isotropic
20220726_d4_10x_RA-18h_F2_163_1787_isotropic
20220726_d4_10x_RA-18h_F2_164_1788_isotropic
20220726_d4_10x_RA-18h_G2_201_1825_isotropic
20220726_d4_10x_RA-18h_G2_202_1826_isotropic
20220726_d4_10x_RA-18h_G2_203_1827_isotropic
20220726_d4_10x_RA-18h_G2_204_1828_isotropic


In [13]:
outDir
df.to_csv(os.path.join(outDir, "wt_FoxA2_Pax6_voxel_counts_ilastikPixelProb_v4.csv"), index=True, header=True)